In [22]:
!pip uninstall -y torch torchvision torchaudio transformers datasets accelerate -q
!pip install --no-cache-dir \
    torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 \
    --index-url https://download.pytorch.org/whl/cpu -q
!pip install --no-cache-dir \
    transformers==4.40.2 datasets==2.19.0 accelerate==0.30.1 \
    scikit-learn pandas joblib -q
print("Installation done – **RESTART KERNEL NOW**")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Installation done – **RESTART KERNEL NOW**


In [23]:
import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"PyTorch {torch.__version__} – Using {device}")

PyTorch 2.3.0 – Using mps


In [24]:
import pandas as pd
df = pd.read_csv("all_messages_clean.csv")
df = df.rename(columns={'Label': 'label'})
df = df.rename(columns={'Text': 'text'})
df['text'] = df['text'].str.lower().str.replace(r'[^a-z0-9\s%₹]', '', regex=True)
print(f"Loaded {len(df)} SMS")
print(df['label'].value_counts())

Loaded 470 SMS
label
Telecom          152
Marketing        114
Banking           70
Shopping          55
Informational     38
Real Estate       29
Spam              12
Name: count, dtype: int64


In [25]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
print(f"Train: {len(train_df)} | Test: {len(test_df)}")

Train: 376 | Test: 94


In [26]:
from transformers import AutoTokenizer
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding=True, max_length=128)

train_ds = Dataset.from_pandas(train_df).map(tokenize, batched=True)
test_ds  = Dataset.from_pandas(test_df).map(tokenize, batched=True)

/Users/kamattap/miniforge3/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 94/94 [00:00<00:00, 13371.70 examples/s]


In [27]:
labels = sorted(df['label'].unique())
label2id = {l:i for i,l in enumerate(labels)}
id2label = {i:l for l,i in label2id.items()}

train_ds = train_ds.map(lambda x: {"labels": label2id[x["label"]]})
test_ds  = test_ds.map(lambda x: {"labels": label2id[x["label"]]})
train_ds = train_ds.remove_columns(["label"])
test_ds  = test_ds.remove_columns(["label"])
print("Labels → integers, string column removed")

Map: 100%|██████████| 94/94 [00:00<00:00, 18939.55 examples/s]

Labels → integers, string column removed


In [28]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
).to(device)

print("Model loaded on", device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded on mps


In [44]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score

args = TrainingArguments(
    output_dir="bert_mps",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
    report_to=[],
    logging_steps=5,
    gradient_accumulation_steps=2,
)

def compute_metrics(eval_pred):
    preds = eval_pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(eval_pred.label_ids, preds),
        "f1": f1_score(eval_pred.label_ids, preds, average="weighted")
    }

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

In [46]:
print("Training on MPS GPU...")
trainer.train()

Training on MPS GPU...


 82%|████████▏ | 376/460 [14:21<03:12,  2.29s/it]

                                            

{'loss': 0.0133, 'grad_norm': 1.8950341939926147, 'learning_rate': 1.9782608695652176e-05, 'epoch': 0.21}


{'loss': 0.1053, 'grad_norm': 0.02738572657108307, 'learning_rate': 1.956521739130435e-05, 'epoch': 0.43}


{'loss': 0.054, 'grad_norm': 0.037173833698034286, 'learning_rate': 1.9347826086956523e-05, 'epoch': 0.64}


...

100%|██████████| 12/12 [00:01<00:00,  8.65it/s]


{'eval_loss': 1.3519965410232544, 'eval_accuracy': 0.7978723404255319, 'eval_f1': 0.8028156054471844, 'eval_runtime': 1.512, 'eval_samples_per_second': 62.171, 'eval_steps_per_second': 7.937, 'epoch': 19.57}



100%|██████████| 460/460 [04:48<00:00,  1.60it/s]

{'train_runtime': 288.3658, 'train_samples_per_second': 26.078, 'train_steps_per_second': 1.595, 'train_loss': 0.04188261352297243, 'epoch': 19.57}


TrainOutput(global_step=460, training_loss=0.04188261352297243, metrics={'train_runtime': 288.3658, 'train_samples_per_second': 26.078, 'train_steps_per_second': 1.595, 'total_flos': 243761747312640.0, 'train_loss': 0.04188261352297243, 'epoch': 19.574468085106382})

In [31]:
print("Training on MPS GPU...")
trainer.train()

Training on MPS GPU...


  2%|▏         | 5/230 [00:03<01:53,  1.97it/s]

{'loss': 0.4097, 'grad_norm': 3.2584807872772217, 'learning_rate': 1.956521739130435e-05, 'epoch': 0.21}


  4%|▍         | 10/230 [00:05<01:37,  2.25it/s]

{'loss': 0.4492, 'grad_norm': 4.471360683441162, 'learning_rate': 1.9130434782608697e-05, 'epoch': 0.43}


...


100%|██████████| 230/230 [02:19<00:00,  2.34it/s]

{'eval_loss': 0.8268794417381287, 'eval_accuracy': 0.7872340425531915, 'eval_f1': 0.7935861705594033, 'eval_runtime': 0.6782, 'eval_samples_per_second': 138.6, 'eval_steps_per_second': 17.694, 'epoch': 9.79}


100%|██████████| 230/230 [02:23<00:00,  1.60it/s]

{'train_runtime': 143.8856, 'train_samples_per_second': 26.132, 'train_steps_per_second': 1.598, 'train_loss': 0.1690822545600974, 'epoch': 9.79}


TrainOutput(global_step=230, training_loss=0.1690822545600974, metrics={'train_runtime': 143.8856, 'train_samples_per_second': 26.132, 'train_steps_per_second': 1.598, 'total_flos': 121880873656320.0, 'train_loss': 0.1690822545600974, 'epoch': 9.787234042553191})

In [47]:
from IPython.display import display

print("EVALUATION RESULTS PER EPOCH:")
for log in trainer.state.log_history:
    if 'eval_accuracy' in log:
        epoch = log.get('epoch', 'N/A')
        acc = log['eval_accuracy']
        print(f"Epoch {epoch}: Accuracy = {acc:.1%}")

EVALUATION RESULTS PER EPOCH:
Epoch 0.9787234042553191: Accuracy = 78.7%
Epoch 2.0: Accuracy = 79.8%
Epoch 2.978723404255319: Accuracy = 79.8%
Epoch 4.0: Accuracy = 77.7%
Epoch 4.9787234042553195: Accuracy = 78.7%
Epoch 6.0: Accuracy = 77.7%
Epoch 6.9787234042553195: Accuracy = 77.7%
Epoch 8.0: Accuracy = 78.7%
Epoch 8.97872340425532: Accuracy = 79.8%
Epoch 10.0: Accuracy = 78.7%
Epoch 10.97872340425532: Accuracy = 78.7%
Epoch 12.0: Accuracy = 79.8%
Epoch 12.97872340425532: Accuracy = 79.8%
Epoch 14.0: Accuracy = 78.7%
Epoch 14.97872340425532: Accuracy = 78.7%
Epoch 16.0: Accuracy = 79.8%
Epoch 16.97872340425532: Accuracy = 79.8%
Epoch 18.0: Accuracy = 79.8%
Epoch 18.97872340425532: Accuracy = 79.8%
Epoch 19.574468085106382: Accuracy = 79.8%


In [48]:
final_metrics = trainer.evaluate()
print(f"\nFINAL TEST ACCURACY: {final_metrics['eval_accuracy']:.1%}")

100%|██████████| 12/12 [00:00<00:00, 18.36it/s]


FINAL TEST ACCURACY: 79.8%


In [49]:
from sklearn.metrics import classification_report
import numpy as np

preds = trainer.predict(test_ds)
pred_labels = np.argmax(preds.predictions, axis=1)
true_labels = preds.label_ids

print(classification_report(true_labels, pred_labels, target_names=labels))

100%|██████████| 12/12 [00:00<00:00, 18.51it/s]

               precision    recall  f1-score   support

      Banking       0.71      0.71      0.71        14
Informational       0.64      0.88      0.74         8
    Marketing       0.81      0.74      0.77        23
  Real Estate       1.00      1.00      1.00         6
     Shopping       0.60      0.82      0.69        11
         Spam       1.00      1.00      1.00         2
      Telecom       0.96      0.80      0.87        30

     accuracy                           0.80        94
    macro avg       0.82      0.85      0.83        94
 weighted avg       0.82      0.80      0.80        94



In [50]:
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
    with torch.no_grad():
        pred = torch.argmax(model(**inputs).logits, dim=1).item()
    return id2label[pred]

print(predict("Get 50% off on recharge!"))           # telecom
print(predict("WIN IPHONE FREE!"))                   # Marketing
print(predict("2 BHK flat in Pune"))                 # real_estate
print(predict("EMI due tomorrow"))                   # banking
print(predict("Buy shoes at 40% off"))               # shopping

Telecom
Marketing
Real Estate
Marketing
Shopping


In [51]:
model.save_pretrained("bert_sms_final")      # ← creates pytorch_model.bin
tokenizer.save_pretrained("bert_sms_final")
import joblib
joblib.dump(id2label, "bert_sms_final/id2label.pkl")

['bert_sms_final/id2label.pkl']